In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
path = 'C:/Nisarg/Projects/AML/'

txt_file = 'Hi-Small_Patterns.txt'
csv_file = 'HI-Small_Trans.csv'

output_csv = 'HI-small_patterns_with_categorical.csv'

In [3]:
df_original = pd.read_csv(path+csv_file)

In [4]:
txt_lines = open(path+txt_file, "r").read()

In [5]:
laundering_patterns = txt_lines.split("BEGIN")

In [6]:
df_original['laundering_pattern'] = ['' for _ in range(len(df_original))]
df_original['ChainID'] = [0 for _ in range(len(df_original))]

In [7]:
trans_dict = {}

for idx, row in tqdm(df_original.iterrows()):
    from_account = row["Account"]
    to_account = row["Account.1"]
    time_stamp = row["Timestamp"]

    trans_dict[f"{from_account}_{to_account}_{time_stamp}"] = idx


5078345it [02:35, 32589.88it/s]


In [12]:
chainID = 0

for pattern in tqdm(laundering_patterns):
    transactions = pattern.split('\n')
    transactions_pattern = transactions[0].split(' ')[-1]
    for trans in transactions[1:]:
        if trans[:3] == 'END':
            break
        from_account = trans.split(',')[2]
        to_account = trans.split(',')[4]
        time_stamp = trans.split(',')[0]
        key = f"{from_account}_{to_account}_{time_stamp}"
        df_original.loc[trans_dict[key], 'laundering_pattern'] = transactions_pattern
        df_original.loc[trans_dict[key], 'ChainID'] = chainID
    chainID += 1     

100%|██████████| 371/371 [00:00<00:00, 623.05it/s]


In [13]:
df_original.to_csv(output_csv)

In [ ]:
output_df = pd.read_csv(output_csv)
print(output_df)

C:\Users\nisar\AppData\Local\Temp\ipykernel_8328\2376663266.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  output_df = pd.read_csv(output_csv)


<bound method NDFrame.describe of          Unnamed: 0         Timestamp  From Bank    Account  To Bank  \
0                 0  2022/09/01 00:20         10  8000EBD30       10   
1                 1  2022/09/01 00:20       3208  8000F4580        1   
2                 2  2022/09/01 00:00       3209  8000F4670     3209   
3                 3  2022/09/01 00:02         12  8000F5030       12   
4                 4  2022/09/01 00:06         10  8000F5200       10   
...             ...               ...        ...        ...      ...   
5078340     5078340  2022/09/10 23:57      54219  8148A6631   256398   
5078341     5078341  2022/09/10 23:35         15  8148A8671   256398   
5078342     5078342  2022/09/10 23:52     154365  8148A6771   256398   
5078343     5078343  2022/09/10 23:46     256398  8148A6311   256398   
5078344     5078344  2022/09/10 23:37     154518  8148A6091   256398   

         Account.1  Amount Received Receiving Currency   Amount Paid  \
0        8000EBD30      3697.